## **1. Installation**

In [ ]:
%tensorflow_version 1.x
!pip install --upgrade h5py==2.10.0
!wget https://pysource.com/extra_files/Mask_RCNN_basic_1.zip
!unzip Mask_RCNN_basic_1.zip
import sys
sys.path.append("/content/Mask_RCNN/mrcnn")
from m_rcnn import *
from visualize import random_colors, get_mask_contours, draw_mask
%matplotlib inline

In [ ]:
!nvidia-smi

In [ ]:
!unzip enhanced_25_annotations.zip

In [ ]:
# Extract Images
images_path = "images.zip"

annotations_path = "/content/result.json"

extract_images(os.path.join("/content/",images_path), "/content/dataset")

In [ ]:
dataset_train = load_image_dataset(os.path.join("/content/", annotations_path), "/content/dataset", "train")
dataset_val = load_image_dataset(os.path.join("/content/", annotations_path), "/content/dataset", "val")
class_number = dataset_train.count_classes()
print('Train: %d' % len(dataset_train.image_ids))
print('Validation: %d' % len(dataset_val.image_ids))
print("Classes: {}".format(class_number))

In [ ]:
# Load image samples
display_image_samples(dataset_train) 

In [ ]:
# Load Configuration
config = CustomConfig(class_number)
#config.display()
model = load_training_model(config)

## **1. Training**


In [ ]:
# Start Training
# This operation might take a long time.
train_head(model, dataset_train, dataset_train, config)

## **2. Run Mask-RCNN on Images for Testing**

You can load here the image and extract the mask using Mask-RCNN


In [ ]:
# Load Image

img = cv2.imread("/content/img0535.png") #Test image


test_model, inference_config = load_inference_model(1, "/content/mask_rcnn_object_0005.h5") #Uplod trained weights
#test_model, inference_config = load_test_model(class_number)


image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Detect results
#r = test_model.detect([image])[0]
#colors = random_colors(80)

######################
# Run detection
results = test_model.detect([image], verbose=1)

colors = random_colors(80)

# Visualize results
r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], r['scores'])


In [ ]:
from google.colab.patches import cv2_imshow
#from matplotlib.patches import Rectangle

# Get Coordinates and show it on the image
boxes = r["rois"]
object_count = len(r["class_ids"])
for i in range(object_count):
    # 1. Mask
    mask = r["masks"][:, :, i]
    contours = get_mask_contours(mask)
    for cnt in contours:
        img = cv2.polylines(img, [cnt], True, colors[i], 2)
        img = draw_mask(img, [cnt], colors[i])

print((object_count))
cv2_imshow(img)

In [ ]:
import numpy as np
import cv2
cv2_imshow(img)
im = cv2.imread("/content/right20025_07_07_22_15_00_Uhr_wbmode1_32.jpg") #Test image
imgray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
ret,thresh = cv2.threshold(imgray,127,255,0)
contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
print(len(contours))
cv2_imshow(cv2.drawContours(im,contours,-1,(0,255,0),3))
cv2_imshow(imgray)

In [ ]:
import cv2

img = cv2.imread("/content/right20025_07_07_22_15_00_Uhr_wbmode1_32.jpg") #Test image
screen_res = 1280, 720
scale_width = screen_res[0] / img.shape[1]
scale_height = screen_res[1] / img.shape[0]
scale = min(scale_width, scale_height)
window_width = int(img.shape[1] * scale)
window_height = int(img.shape[0] * scale)

#Shrimps = 25
position = (10,40)

cv2.resize(img, (window_width, window_height))
cv2.putText(img, str(object_count), position, cv2.FONT_HERSHEY_SIMPLEX,1, (0,255,0),2) #(209, 80, 0, 255),3)
cv2.imwrite("/content/Op_frame434.png",img)

True

In [ ]:
mask = r['masks']
mask = mask.astype(int)
mask.shape
# 17 in the output means how many Shrimps were detected

(544, 960, 14)

In [ ]:
for i in range(mask.shape[2]):
    temp = cv2.imread('/content/right20025_07_07_22_15_00_Uhr_wbmode1_32.jpg')
  
    path = "/content/output"+str(i)+".jpg" ## create file output in the workspace
    
    for j in range(temp.shape[2]):
        temp[:,:,j] = temp[:,:,j] * mask[:,:,i]
        #temp[:,:,j] = temp[0,255,j] * mask[:,:,i]
       
    cv2.imwrite(path,temp)
    cv2_imshow(temp)
    print(temp.shape[2])




## length calculations

In [ ]:
import cv2
import numpy as np
import os
import math
import glob
from google.colab.patches import cv2_imshow


def length_cal(Path):
   # Load image, grayscale, Gaussian blur, threshold
    image = cv2.imread(Path)

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (3,3), 0)
    thresh = cv2.threshold(blur,10, 255, cv2.THRESH_BINARY)[1]

    #(thresh, blackAndWhiteImage) = cv2.threshold(blur, 115, 255 ,cv2.THRESH_BINARY_INV) 

    # Find contours
    cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    c = max(cnts, key=cv2.contourArea)

    # Obtain outer coordinates
    left = tuple(c[c[:, :, 0].argmin()][0])
    right = tuple(c[c[:, :, 0].argmax()][0])
    top = tuple(c[c[:, :, 1].argmin()][0])
    bottom = tuple(c[c[:, :, 1].argmax()][0])


    cv2_imshow(thresh)
    cv2_imshow(thresh)
    cv2.waitKey()
    cv2.imwrite(f'/content/thres/{Path}.png', thresh) 

    #import matplotlib.pyplot as plt
    #im =  plt.imread(r'/content/download.png')
    #implot = plt.imshow(im)
    # put a blue dot at (10, 20)
    #plt.scatter(left[0], left[1],s=0.5)
    #plt.scatter(right[0], right[1],s=0.5)
    #plt.scatter(top[0], top[1],s=0.5)
    #plt.scatter(bottom[0], bottom[1],s=0.5)
    
  
    #plt.savefig(r'/content/outputwithoutput.png',dpi=300)

    #plt.show()

    ListOfCoordinates = []
    ListOfCoordinates.append(left)
    ListOfCoordinates.append(right)
    ListOfCoordinates.append(top)
    ListOfCoordinates.append(bottom)

    print(ListOfCoordinates)
    def distance(p1, p2):
        return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

    dist_cordinates = []

    for i in range(3):

       (dist_cordinates.append(distance(ListOfCoordinates[0], ListOfCoordinates[i+1])))    

    (dist_cordinates.append(distance(ListOfCoordinates[1], ListOfCoordinates[2])))
    (dist_cordinates.append(distance(ListOfCoordinates[1], ListOfCoordinates[3])))
    (dist_cordinates.append(distance(ListOfCoordinates[2], ListOfCoordinates[3])))
    
    return(max(dist_cordinates )* 0.174)##  0.1176



In [ ]:
length_list= []
#x = np.empty[]

for img in glob.glob('/content/output*.jpg'):
    
    l = length_cal(img)
    print(l)
    length_list.append(l)

print(length_list)
print("max length", max(length_list))
def average(lst):
  return sum(lst) / len(lst)

average_length = average(length_list)


print("Average length = ",round(average_length, 2))

In [ ]:
import cv2

img = cv2.imread("/content/output.png")
screen_res = 1280, 720
scale_width = screen_res[0] / img.shape[1]
scale_height = screen_res[1] / img.shape[0]
scale = min(scale_width, scale_height)
window_width = int(img.shape[1] * scale)
window_height = int(img.shape[0] * scale)

cv2.resize(img, (window_width, window_height))
 
# Using cv2.putText()
new_image = cv2.putText(
  img = img,
  text = "Average length ={:.2f}".format(average_length),
  org = (10,40),
  fontFace = cv2.FONT_HERSHEY_DUPLEX,
  fontScale = 1.0,
  color = (0, 255, 0),
  thickness = 2
)
 
# Saving the new image
cv2.imwrite("/content/length.png", new_image)

In [ ]:
import cv2
import numpy as np
import os
import math
import glob
from google.colab.patches import cv2_imshow
!pip install imutils
!pip install --upgrade imutils
import imutils
from scipy.spatial import distance as dist
from imutils import perspective
from imutils import contours
import numpy as np
import argparse



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# sorting the contours function
def sort_contours(cnts, method="left-to-right"):
	# initialize the reverse flag and sort index
	reverse = False
	i = 0
	# handle if we need to sort in reverse
	if method == "right-to-left" or method == "bottom-to-top":
		reverse = True
	# handle if we are sorting against the y-coordinate rather than
	# the x-coordinate of the bounding box
	if method == "top-to-bottom" or method == "bottom-to-top":
		i = 1
	# construct the list of bounding boxes and sort them from top to
	# bottom
	boundingBoxes = [cv2.boundingRect(c) for c in cnts]
	(cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
		key=lambda b:b[1][i], reverse=reverse))
	# return the list of sorted contours and bounding boxes
	return (cnts, boundingBoxes)

In [ ]:
#finding mid point function

def midpoint(ptA, ptB):
	return ((ptA[0] + ptB[0]) * 0.5, (ptA[1] + ptB[1]) * 0.5)

In [ ]:
# construct the argument parse and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--image", required=True,
	help="path to the input image")
ap.add_argument("-w", "--width", type=float, required=True,
	help="width of the left-most object in the image (in inches)")
args = vars(ap.parse_args())

In [ ]:
def length_cal(Path):
   # Load image, grayscale, Gaussian blur, threshold
    image = cv2.imread(Path)

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (3,3), 0)
    thresh = cv2.threshold(blur,10, 255, cv2.THRESH_BINARY)[1]

    #(thresh, blackAndWhiteImage) = cv2.threshold(blur, 115, 255 ,cv2.THRESH_BINARY_INV) 

    
# find contours in the edge map
cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL,
	cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
# sort the contours from left-to-right and initialize the
# 'pixels per metric' calibration variable
(cnts, _) = contours.sort_contours(cnts)
pixelsPerMetric = None

In [ ]:
# loop over the contours individually
for c in cnts:
	# if the contour is not sufficiently large, ignore it
	if cv2.contourArea(c) < 100:
		continue
	# compute the rotated bounding box of the contour
	orig = image.copy()
	box = cv2.minAreaRect(c)
	box = cv2.cv.BoxPoints(box) if imutils.is_cv2() else cv2.boxPoints(box)
	box = np.array(box, dtype="int")
	# order the points in the contour such that they appear
	# in top-left, top-right, bottom-right, and bottom-left
	# order, then draw the outline of the rotated bounding
	# box
	box = perspective.order_points(box)
	cv2.drawContours(orig, [box.astype("int")], -1, (0, 255, 0), 2)
	# loop over the original points and draw them
	for (x, y) in box:
		cv2.circle(orig, (int(x), int(y)), 5, (0, 0, 255), -1)

In [ ]:
# unpack the ordered bounding box, then compute the midpoint
	# between the top-left and top-right coordinates, followed by
	# the midpoint between bottom-left and bottom-right coordinates
(tl, tr, br, bl) = box
(tltrX, tltrY) = midpoint(tl, tr)
(blbrX, blbrY) = midpoint(bl, br)
	# compute the midpoint between the top-left and top-right points,
	# followed by the midpoint between the top-righ and bottom-right
(tlblX, tlblY) = midpoint(tl, bl)
(trbrX, trbrY) = midpoint(tr, br)
	# draw the midpoints on the image
cv2.circle(orig, (int(tltrX), int(tltrY)), 5, (255, 0, 0), -1)
cv2.circle(orig, (int(blbrX), int(blbrY)), 5, (255, 0, 0), -1)
cv2.circle(orig, (int(tlblX), int(tlblY)), 5, (255, 0, 0), -1)
cv2.circle(orig, (int(trbrX), int(trbrY)), 5, (255, 0, 0), -1)
	# draw lines between the midpoints
cv2.line(orig, (int(tltrX), int(tltrY)), (int(blbrX), int(blbrY)),
	(255, 0, 255), 2)
cv2.line(orig, (int(tlblX), int(tlblY)), (int(trbrX), int(trbrY)),
	(255, 0, 255), 2)

In [ ]:
# compute the Euclidean distance between the midpoints
dA = dist.euclidean((tltrX, tltrY), (blbrX, blbrY))	
dB = dist.euclidean((tlblX, tlblY), (trbrX, trbrY))
	# if the pixels per metric has not been initialized, then
	# compute it as the ratio of pixels to supplied metric
	# (in this case, inches)
if pixelsPerMetric is None:
	pixelsPerMetric = dB / args["width"]

In [ ]:
# compute the size of the object
	dimA = dA / pixelsPerMetric
	dimB = dB / pixelsPerMetric
	# draw the object sizes on the image
	cv2.putText(orig, "{:.1f}in".format(dimA),
		(int(tltrX - 15), int(tltrY - 10)), cv2.FONT_HERSHEY_SIMPLEX,
		0.65, (255, 255, 255), 2)
	cv2.putText(orig, "{:.1f}in".format(dimB),
		(int(trbrX + 10), int(trbrY)), cv2.FONT_HERSHEY_SIMPLEX,
		0.65, (255, 255, 255), 2)
	# show the output image
	cv2.imshow("Image", orig)
	cv2.waitKey(0)